# Coursera Capstone Final Project

I want to open a new Restaurant in Porto District with the district code in Portugal of 4. So I want to decide which is the best municipality place to open the new restaurant. My criteria is based on the amount of restaurants in each municipality vs. population density in each municipality.

In [31]:
#import the packages
import pandas as pd

In [32]:
#creates a source for the Beautiful Soup
df = pd.read_excel(r'pordata.xlsx') # Taken from www.pordata.pt
df = df[['Region','Municipality','Density']]
# Add Postal code , latitude and longitude collumn
#df["MCode"] = ""
df["Latitude"] = ""
df["Longitude"] = ""

df.head()

,Region,Municipality,Density,Latitude,Longitude
0,Alto Minho,Arcos de Valdevez,47,,
1,Alto Minho,Caminha,116.4,,
2,Alto Minho,Melgaço,34.4,,
3,Alto Minho,Monção,85.1,,
4,Alto Minho,Paredes de Coura,62.1,,


In [33]:
# All districts and municipalities of Portugal
code_data = pd.read_csv("concelhos.csv")[["nome_concelho","cod_distrito","cod_concelho"]] # taken from http://centraldedados.pt/codigos_postais/
code_data.columns = ['Municipality', 'DCode', 'MCode']

code_data.head()

,Municipality,DCode,MCode
0,Castelo de Paiva,1,6
1,Espinho,1,7
2,Estarreja,1,8
3,Santa Maria da Feira,1,9
4,Ílhavo,1,10


In [34]:
# Porto district code is 4
porto_data = code_data[code_data['DCode'] == 4]
porto_data.head()

,Municipality,DCode,MCode
47,Miranda do Douro,4,6
48,Mirandela,4,7
49,Mogadouro,4,8
50,Torre de Moncorvo,4,9
51,Vila Flor,4,10


In [35]:
# Copy MCode to df
dfporto = df.merge(porto_data[["Municipality", "MCode"]])

dfporto.head()

,Region,Municipality,Density,Latitude,Longitude,MCode
0,Douro,Carrazeda de Ansiães,20.4,,,3
1,Douro,Freixo de Espada à Cinta,13.7,,,4
2,Douro,Torre de Moncorvo,14.6,,,9
3,Terras de Trás-os-Montes,Alfândega da Fé,14.2,,,1
4,Terras de Trás-os-Montes,Bragança,28.7,,,2
